In [5]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("yasserh/twitter-tweets-sentiment-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/twitter-tweets-sentiment-dataset


In [6]:
import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict, load_dataset
from transformers import (
AutoTokenizer,
AutoModelForSequenceClassification,
TrainingArguments,
Trainer,
DataCollatorWithPadding
)
import torch
from sklearn.metrics import classification_report, accuracy_score
sns.set()

2025-06-24 12:36:30.174652: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750768590.405279      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750768590.467065      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
os.listdir('/kaggle/input/twitter-tweets-sentiment-dataset')

['Tweets.csv']

In [8]:
df = pd.read_csv('/kaggle/input/twitter-tweets-sentiment-dataset/Tweets.csv')
df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [9]:
df.shape

(27481, 4)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27481 entries, 0 to 27480
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   textID         27481 non-null  object
 1   text           27480 non-null  object
 2   selected_text  27480 non-null  object
 3   sentiment      27481 non-null  object
dtypes: object(4)
memory usage: 858.9+ KB


In [11]:
df.isnull().sum()

textID           0
text             1
selected_text    1
sentiment        0
dtype: int64

In [12]:
df.dropna(inplace = True)

In [13]:
df.apply(pd.isnull).sum()

textID           0
text             0
selected_text    0
sentiment        0
dtype: int64

In [14]:
df['sentiment'].value_counts()

sentiment
neutral     11117
positive     8582
negative     7781
Name: count, dtype: int64

In [15]:
df0 = df[df['sentiment'] == 'neutral'].sample(7700, random_state = 101)
df1 = df[df['sentiment'] == 'positive'].sample(7700, random_state = 101)
df2 = df[df['sentiment'] == 'negative'].sample(7700, random_state = 101)

In [16]:
df_b = pd.concat([df0, df1, df2])

In [17]:
df_b.head()

,textID,text,selected_text,sentiment
8504,e557f77a4d,not letting anyone get me down tired of takin...,not letting anyone get me down tired of takin...,neutral
22082,0064be4ccd,Ahh its so gloomy out I`m pretty sure I just ...,Ahh its so gloomy out I`m pretty sure I just ...,neutral
952,ed5a6ba312,My mom just texted me and told me that Rodney ...,My mom just texted me and told me that Rodney ...,neutral
15632,d74c09c180,part 2: social networking??.. there is even r...,part 2: social networking??.. there is even ro...,neutral
13453,7af009703a,"no idea offhand, but if you keep tweeting abo...","no idea offhand, but if you keep tweeting abou...",neutral


In [18]:
df_b['sentiment'] = df_b['sentiment'].replace({'neutral': 0, 'negative': 1, 'positive': 2})

/tmp/ipykernel_35/1584630402.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_b['sentiment'] = df_b['sentiment'].replace({'neutral': 0, 'negative': 1, 'positive': 2})


In [19]:
df_b.head()

,textID,text,selected_text,sentiment
8504,e557f77a4d,not letting anyone get me down tired of takin...,not letting anyone get me down tired of takin...,0
22082,0064be4ccd,Ahh its so gloomy out I`m pretty sure I just ...,Ahh its so gloomy out I`m pretty sure I just ...,0
952,ed5a6ba312,My mom just texted me and told me that Rodney ...,My mom just texted me and told me that Rodney ...,0
15632,d74c09c180,part 2: social networking??.. there is even r...,part 2: social networking??.. there is even ro...,0
13453,7af009703a,"no idea offhand, but if you keep tweeting abo...","no idea offhand, but if you keep tweeting abou...",0


In [20]:
X = df_b['text']
y = df_b['sentiment']

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 101, stratify = y)

In [22]:
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size = 0.4, random_state = 101, stratify = y_test)

In [23]:
print(f'Training set size: {X_train.shape[0]}')
print(f'Test set size: {X_test.shape[0]}')
print(f'Validation set size: {X_val.shape[0]}')

Training set size: 16170
Test set size: 4158
Validation set size: 2772


In [24]:
df_train = pd.DataFrame({
    'text': X_train,
    'labels': y_train
})

df_test = pd.DataFrame({
    'text': X_test,
    'labels': y_test
})

df_val = pd.DataFrame({
    'text': X_val,
    'labels': y_val
})

In [25]:
train_dict = DatasetDict({'train': Dataset.from_pandas(df_train)})

In [26]:
train_dict

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', '__index_level_0__'],
        num_rows: 16170
    })
})

In [27]:
test_dict = DatasetDict({'test': Dataset.from_pandas(df_test)})

In [28]:
val_dict = DatasetDict({'val': Dataset.from_pandas(df_val)})

In [29]:
train_dict = train_dict.remove_columns('__index_level_0__')
test_dict = test_dict.remove_columns('__index_level_0__')
val_dict = val_dict.remove_columns('__index_level_0__')

In [30]:
model_path = 'google-bert/bert-base-uncased'

In [31]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [32]:
id2label = {0: 'neutral', 1: 'negative', 2: 'positive'}
label2id = {'neutral': 0, 'negative': 1, 'positive': 2}

In [33]:
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation = True)

In [34]:
tokenized_train = train_dict.map(preprocess_function, batched = True)
tokenized_test = test_dict.map(preprocess_function, batched = True)
tokenized_val = val_dict.map(preprocess_function, batched = True)

Map:   0%|          | 0/16170 [00:00<?, ? examples/s]

Map:   0%|          | 0/4158 [00:00<?, ? examples/s]

Map:   0%|          | 0/2772 [00:00<?, ? examples/s]

In [35]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_path,
    num_labels = 3,
    id2label = id2label,
    label2id = label2id
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
for name, param in model.base_model.named_parameters():
    param.requires_grad = False

In [37]:
train_last_n_layers = 3
total_layers = 12  # BERT-base has 12 layers

for name, param in model.base_model.named_parameters():
    # default no training
    param.requires_grad = False
    # check if in last N layers or pooler
    for layer_num in range(total_layers - train_last_n_layers, total_layers):
        if f'encoder.layer.{layer_num}' in name or 'pooler' in name:
            param.requires_grad = True
            break

In [38]:
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

In [39]:
lr = 2e-5
batch_size = 8
num_epochs = 3

In [40]:
training_args = TrainingArguments(
    output_dir = '/kaggle/working/',
    learning_rate = lr,
    per_device_eval_batch_size = batch_size,
    num_train_epochs = num_epochs,
    logging_strategy = 'epoch',
    eval_strategy = 'epoch',
    save_strategy = 'epoch',
    load_best_model_at_end = True,
    report_to = 'tensorboard'
)

In [41]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_train['train'],
    eval_dataset = tokenized_val['val'],
    tokenizer = tokenizer,
    data_collator = data_collator
)

/tmp/ipykernel_35/1639913444.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [42]:
print('Start training...')
trainer.train()
print('Training finish...')

Start training...


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.697200,0.572158
2,0.554000,0.552339
3,0.512000,0.553177


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Training finish...


In [43]:
trainer.save_model('/kaggle/working/sentiment_analysis')
tokenizer.save_pretrained('/kaggle/working/sentiment_analysis')

('/kaggle/working/sentiment_analysis/tokenizer_config.json',
 '/kaggle/working/sentiment_analysis/special_tokens_map.json',
 '/kaggle/working/sentiment_analysis/vocab.txt',
 '/kaggle/working/sentiment_analysis/added_tokens.json',
 '/kaggle/working/sentiment_analysis/tokenizer.json')

In [44]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [45]:
def predict_sentiment(text):
    device = next(model.parameters()).device 
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()} 
    
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class_id = torch.argmax(logits, dim=1).item()
    
    id2label = {0: 'neutral', 1: 'negative', 2: 'positive'}
    return id2label[predicted_class_id]

In [46]:
prediction_output = trainer.predict(tokenized_val['val'])
logits = prediction_output.predictions
predicted_labels = np.argmax(logits, axis = 1)
true_labels = prediction_output.label_ids
acc = accuracy_score(true_labels, predicted_labels)
acc

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


0.7705627705627706